In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
from decouple import config


HPC_USER = config('HPC_USER')
HPC_HOST = config('HPC_HOST')
HPC_PASSPHRASE = config('HPC_PASSPHRASE')

In [ ]:
from pathlib import Path

from math_rag.infrastructure.clients import PBSProClient, SFTPClient


sftp_client = SFTPClient(HPC_HOST, HPC_USER, HPC_PASSPHRASE)
pbs_pro_client = PBSProClient(HPC_HOST, HPC_USER, HPC_PASSPHRASE)

In [ ]:
# TODO remove .. when not in /notebooks
source = Path('../assets/huggingface/tgi/tgi.sh')
target = Path('/lustre/home/lpanic/tgi_test.sh')

await sftp_client.upload(source, target)

In [ ]:
# TODO remove .. when not in /notebooks
source = Path('/lustre/home/lpanic/tgi_test.sh')
target = Path('../tmp/tgi.sh')

await sftp_client.download(source, target)

In [ ]:
res = await pbs_pro_client.queue_state()
print(res)

In [ ]:
# limited to 80 characters (79 + newline)
len('    Error_Path = x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/hello_world')

79

In [2]:
# qstat -f 465317
queue_status = """
Job Id: 465317.x3000c0s25b0n0.hsn.hpc.srce.hr
    Job_Name = hello
    Job_Owner = lpanic@x3000c0s27b0n0.hsn.hpc.srce.hr
    resources_used.cpupercent = 0
    resources_used.cput = 00:00:00
    resources_used.mem = 0b
    resources_used.ncpus = 1
    resources_used.vmem = 0kb
    resources_used.walltime = 00:00:00
    job_state = R
    queue = cpu-single
    server = x3000c0s25b0n0.hsn.hpc.srce.hr
    Checkpoint = u
    ctime = Fri Mar 28 10:45:48 2025
    Error_Path = x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/hello_world
	/hello.e465317
    exec_host = x8000c1s4b0n1/1
    exec_vnode = (x8000c1s4b0n1:mem=1843200kb:ncpus=1:ngpus=0)
    Hold_Types = n
    Join_Path = n
    Keep_Files = oed
    Mail_Points = a
    mtime = Fri Mar 28 10:45:53 2025
    Output_Path = x3000c0s27b0n0.hsn.hpc.srce.hr:/lustre/home/lpanic/hello_worl
	d/hello.o465317
    Priority = 0
    qtime = Fri Mar 28 10:45:48 2025
    Rerunable = True
    Resource_List.mem = 1800mb
    Resource_List.ncpus = 1
    Resource_List.ngpus = 0
    Resource_List.nodect = 1
    Resource_List.place = pack
    Resource_List.select = 1:mem=1800mb:ncpus=1:ngpus=0
    Resource_List.walltime = 48:00:00
    stime = Fri Mar 28 10:45:48 2025
    session_id = 3900810
    jobdir = /lustre/home/lpanic
    substate = 42
    Variable_List = PBS_O_HOME=/lustre/home/lpanic,PBS_O_LANG=en_US.UTF-8,
	PBS_O_LOGNAME=lpanic,
	PBS_O_PATH=/lustre/home/lpanic/.local/bin:/lustre/home/lpanic/bin:/opt
	/clmgr/sbin:/opt/clmgr/bin:/opt/sgi/sbin:/opt/sgi/bin:/usr/share/Module
	s/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/c3/bin:/op
	t/pbs/bin:/sbin:/bin,PBS_O_MAIL=/var/spool/mail/lpanic,
	PBS_O_SHELL=/bin/bash,PBS_O_HOST=x3000c0s27b0n0.hsn.hpc.srce.hr,
	PBS_O_WORKDIR=/lustre/home/lpanic/hello_world,PBS_O_SYSTEM=Linux,
	PBS_O_QUEUE=RouteQ
         = Job run at Fri Mar 28 at 10:45 on (x8000c1s4b0n1:mem=1843200kb:nc
	pus=1:ngpus=0)
    etime = Fri Mar 28 10:45:48 2025
    run_count = 1
    eligible_time = 00:00:05
    Submit_arguments = -koed hello.sh
    project = _pbs_project_default
    Submit_Host = x3000c0s27b0n0.hsn.hpc.srce.hr
"""

In [ ]:
from math_rag.infrastructure.models.hpcs.pbs import PBSProJob


job = PBSProJob.from_queue_status(queue_status)

In [ ]:
"""
[lpanic@x3000c0s27b0n0 hello_world]$ qsub hello2.sh
465698.x3000c0s25b0n0.hsn.hpc.srce.hr
[lpanic@x3000c0s27b0n0 hello_world]$ qstat 465698.x3000c0s25b0n0.hsn.hpc.srce.hr
Job id                 Name             User              Time Use S Queue
---------------------  ---------------- ----------------  -------- - -----
465698.x3000c0s25b0n0  hello            lpanic                   0 Q gpu
"""

In [ ]:
"""
qstat -x 465698 | awk 'NR==3 {print $1, $2, $3, $4, $5, $6}'
"""

"""
465698.x3000c0s25b0n0 hello lpanic 0 Q gpu
"""

In [ ]:
"""
gpustat
Job             Node            GPU        Gpu-ut     Memory used
-----------------------------------------------------------------
465711          x8000c2s0b0n1   GPU-3      0 %        1 MiB
"""

In [ ]:
"""
job_tmp_size

JOB_ID : 465711
4.0K /lustre/scratch/pbs.465711.x3000c0s25b0n0.hsn.hpc.srce.hr
"""

In [ ]:
"""
NOTE jobstat and jobstat --failed have the same format
jobstat --failed

JOB ID     NCPUS      LOAD %     MEM        USED.MEM   WALLTIME   USED.WALLTIME   Exit
--------------------------------------------------------------------------------------
465709     4          0          1gb        0gb        48:00:00   00:00:21        271x
"""

In [ ]:
"""
qlive
----------------------------
Table is showing free/total cpus/gpus per queue, fully free and unavailable nodes
----------------------------
          Queue      CPUs (f/t)      GPUs (f/t) Fully Free Nodes Unavailable Nodes
bigmem                  256/256             0/0              2/2               0/2
cpu                    428/6400             0/0             0/50              0/50
cpu-radionica           256/256             0/0              2/2               0/2
cpu-single              127/384             0/0              0/3               0/3
cpu-test               428/6400             0/0             0/50              0/50
gpu                    347/1280            2/80             0/20              0/20
gpu-radionica           128/128             8/8              2/2               0/2
gpu-test                  0/128             0/8              0/2               0/2
login-cpu               128/128             0/0              1/1               0/1
login-gpu                 52/52             1/1              1/1               0/1
"""

In [ ]:
"""
qlive | awk 'NR>=4 {print $1, $2, $3, $4, $5}'
Queue CPUs (f/t) GPUs (f/t)
bigmem 256/256 0/0 2/2 0/2
cpu 428/6400 0/0 0/50 0/50
cpu-radionica 256/256 0/0 2/2 0/2
cpu-single 127/384 0/0 0/3 0/3
cpu-test 428/6400 0/0 0/50 0/50
gpu 347/1280 2/80 0/20 0/20
gpu-radionica 128/128 8/8 2/2 0/2
gpu-test 0/128 0/8 0/2 0/2
login-cpu 128/128 0/0 1/1 0/1
login-gpu 52/52 1/1 1/1 0/1
"""